# Usage Example

In [1]:
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import re

###############################################
# Assuming this script is outside the model dir
###############################################

model_checkpoint_path = '../downloaded_from_HF/EduRABSA_SLM_v1_SLERP_phi4mini'
# model_checkpoint_path = '../downloaded_from_HF/EduRABSA_SLM_v1_SLERP_qw2.5-1.5B'

# inference config for evaluation
generation_args = {"max_new_tokens": 1024,
                   "return_full_text": False,
                   "temperature": None, # or a float after setting 'do_sample=True' 
                   "top_p": None,
                   "top_k": None,
                   "do_sample": False
                   }

model_kwargs = {
    'use_cache': False,
    'trust_remote_code': False,  # or True if you haven't downloaded the model to local
    'attn_implementation': 'flash_attention_2',
    'torch_dtype': torch.bfloat16,  # loaded pre-trained model torch dtype
    'device_map': {"": 0} # or 'cpu',  
}

accelerator = Accelerator()
device = accelerator.device

###############################################
# Load tokenizer 
###############################################
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_path)
tokenizer.model_max_length = generation_args['max_new_tokens']

###############################################
# Load pre-trained model & set up trainer
###############################################

tokenizer.padding_side = 'left' 

merged_model = AutoModelForCausalLM.from_pretrained(model_checkpoint_path, **model_kwargs)

# move model to GPU 
merged_model = merged_model.to(device) 

###############################################
# Create pipeline for inference
###############################################

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, torch_dtype=model_kwargs['torch_dtype']) 

pipeline([{"role": "user", "content": "hi"}], **generation_args)[0]['generated_text']

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


'Hello! How can I assist you today?'

In [2]:
###############################################
# Mini-test on ABSA tasks
# Please check the GitHub repo for all prompts
# used for training
###############################################

instruction = """### Instruction:
Given the input text, extract ALL pairs of opinion expressions and their corresponding aspect terms about the course, staff, or university. Then classify the category and sentiment for each aspect-opinion pair.
Opinion expressions are words/phrases expressing evaluation, feeling, or judgment (including both explicit and implicit opinions, not objective facts).
Aspect terms are opinion targets. Only use a pronoun if you cannot find a direct aspect term in the same sentence or adjacent context. 
Each aspect-opinion-category-sentiment combination is a quadruplet. 

**Rules:**
- Extract EVERY opinion in the text, including both explicit and implicit opinion expressions.
- Extract all opinion and aspect terms VERBATIM and as CONSECUTIVE tokens. 
- Use 'null' for implicit aspects. Opinions cannot be null.
- If an aspect is mapped to multiple opinion expressions, or vice versa, extract each 1:1 pair separately. 
- Categorise each aspect-opinion pair first into one main category (the keys) in the category_mapping below, and then into one of its appropriate subcategories (values for the key). The category label follows "Main category - subcategory" format.
category_mapping = {
  "Course": ["Content", "Learning activity", "Assessment", "Workload", "Difficulty", "Course materials", "Technology & tools", "Overall"],
  "Staff": ["Teaching", "Knowledge & skills", "Helpfulness", "Attitude", "Personal traits", "Overall"],
  "University": ["Cost", "Opportunities", "Programme", "Campus & facilities", "Culture & diversity", "Information & Services", "Social engagement & activities", "Overall"]
}

- Classify the sentiment into one of 'positive', 'neutral', 'negative'. 
- Use these specific tags for each component within each quadruplet: <asp>aspect terms</asp>, <opn>opinion expressions</opn>, <cat>category</cat>, <sen>sentiment</sen>

**Critical formatting requirements:**
- Output MUST be a valid Python list
- Quadruplets MUST be separated by commas

**Output format:** 
[<asp>...</asp><opn>...</opn><cat>...</cat><sen>...</sen>, <asp>...</asp><opn>...</opn><cat>...</cat><sen>...</sen>, ..., <asp>...</asp><opn>...</opn><cat>...</cat><sen>...</sen>]

### Examples:
Input: "The professor was knowledgeable but the assignments were too hard."
Output: [<asp>professor</asp><opn>knowledgeable</opn><cat>Staff - Knowledge & skills</cat><sen>positive</sen>, <asp>assignments</asp><opn>too hard</opn><cat>Course - Assessment</cat><sen>negative</sen>]

Input: "It was disappointing overall."
Output: [<asp>null</asp><opn>disappointing</opn><cat>Course - Overall</cat><sen>negative</sen>]

Input: "She never reply to emails or answer questions"
Output: [<asp>She</asp><opn>never reply to emails or answer questions</opn><cat>Staff - Helpfulness</cat><sen>negative</sen>]

Input: "There were 10 assignments, 5 quizzes, 1 final exam."
Output: [<asp></asp><opn></opn><cat></cat><sen></sen>]
"""

system_prompt = "You are an expert in aspect-based sentiment analysis."

test_input = "The course itself was OK, but the lectures needed better organisation. The final assignment was waaaay too hard IMO, but what do I know I only got a B- ..."

user_content = f"""## Task type: ASQE
{instruction}
## Input text:
{test_input}
"""

messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content}]


def parse_output(output: str) -> list:
    """Parse ABSA output into DataFrame."""
    output = output.strip().strip('[]').strip()
    quadruplets = re.split(r'(?=<asp>)', output)
    quadruplets = [q.strip().rstrip(',').strip() for q in quadruplets if q.strip()]

    results = []
    for quad in quadruplets:
        aspect_match = re.search(r'<asp>(.*?)</asp>', quad)
        opinion_match = re.search(r'<opn>(.*?)</opn>', quad)
        category_match = re.search(r'<cat>(.*?)</cat>', quad)
        sentiment_match = re.search(r'<sen>(.*?)</sen>', quad)

        if all([aspect_match, opinion_match, category_match, sentiment_match]):
            results.append({
                'aspect': aspect_match.group(1).strip(),
                'opinion': opinion_match.group(1).strip(),
                'category': category_match.group(1).strip(),
                'sentiment': sentiment_match.group(1).strip()
            })
    return results

In [3]:
raw_model_output  = pipeline(messages, **generation_args)[0]['generated_text']

output = parse_output(raw_model_output)

print(test_input, '\n')

output

The course itself was OK, but the lectures needed better organisation. The final assignment was waaaay too hard IMO, but what do I know I only got a B- ... 



[{'aspect': 'course',
  'opinion': 'OK',
  'category': 'Course - Overall',
  'sentiment': 'neutral'},
 {'aspect': 'lectures',
  'opinion': 'needed better organisation',
  'category': 'Course - Learning activity',
  'sentiment': 'negative'},
 {'aspect': 'final assignment',
  'opinion': 'waaaay too hard',
  'category': 'Course - Assessment',
  'sentiment': 'negative'}]